In [1]:
import numpy as np
import pandas as pd 



In [2]:
use_cols = ["BUYER_STATE","BUYER_COUNTY", "CALC_BASE_WT_IN_GM", "MME_Conversion_Factor","TRANSACTION_DATE", "DRUG_NAME"]





In [3]:
import pandas as pd
df = pd.read_csv('/Users/dapoadegbile/Desktop/720Python/arcos-fl-statewide-itemized.csv.gz',  compression='gzip', usecols = use_cols,
                chunksize = 1000000,
                error_bad_lines=False)

In [4]:
df

## Shipment data in a dataframe

In [5]:
df1 = pd.DataFrame()
for i in df:
    a = pd.DataFrame(i)
    df1 = df1.append(a)

# for d,i in df.iterrows():
#     print(d)
#     break


In [6]:
df1

,BUYER_STATE,BUYER_COUNTY,DRUG_NAME,TRANSACTION_DATE,CALC_BASE_WT_IN_GM,MME_Conversion_Factor
0,FL,PALM BEACH,OXYCODONE,8182006,3.5860,1.5
1,FL,PALM BEACH,OXYCODONE,11292006,7.1720,1.5
2,FL,PALM BEACH,OXYCODONE,2062007,21.5160,1.5
3,FL,PALM BEACH,OXYCODONE,3012007,21.5160,1.5
4,FL,PALM BEACH,OXYCODONE,4162007,5.3790,1.5
...,...,...,...,...,...,...
15104978,FL,HILLSBOROUGH,OXYCODONE,12042008,268.9500,1.5
15104979,FL,HILLSBOROUGH,HYDROCODONE,8132009,3.0270,1.0
15104980,FL,HILLSBOROUGH,HYDROCODONE,12302009,1.5135,1.0
15104981,FL,HILLSBOROUGH,OXYCODONE,3182010,67.2375,1.5


## Cleaning shipment data frame

In [7]:
df1['TRANSACTION_YEAR'] = df1['TRANSACTION_DATE'].astype(str).str[-4:]


In [8]:
df1['CALC_BASE_WT_IN_MG'] = df1['CALC_BASE_WT_IN_GM'] * 1000

In [9]:
df1['MORPHINE_EQUIV_IN_MG'] = df1['CALC_BASE_WT_IN_MG'] * df1['MME_Conversion_Factor']

final_shipments_FL = df1[["TRANSACTION_YEAR", "BUYER_STATE", "BUYER_COUNTY", "MORPHINE_EQUIV_IN_MG"]]
final_shipments_FL



,TRANSACTION_YEAR,BUYER_STATE,BUYER_COUNTY,MORPHINE_EQUIV_IN_MG
0,2006,FL,PALM BEACH,5379.00
1,2006,FL,PALM BEACH,10758.00
2,2007,FL,PALM BEACH,32274.00
3,2007,FL,PALM BEACH,32274.00
4,2007,FL,PALM BEACH,8068.50
...,...,...,...,...
15104978,2008,FL,HILLSBOROUGH,403425.00
15104979,2009,FL,HILLSBOROUGH,3027.00
15104980,2009,FL,HILLSBOROUGH,1513.50
15104981,2010,FL,HILLSBOROUGH,100856.25


In [10]:
final_shipments_FL = final_shipments_FL.rename(columns={"BUYER_COUNTY": "County", "TRANSACTION_YEAR":"Year"})
final_shipments_FL

,Year,BUYER_STATE,County,MORPHINE_EQUIV_IN_MG
0,2006,FL,PALM BEACH,5379.00
1,2006,FL,PALM BEACH,10758.00
2,2007,FL,PALM BEACH,32274.00
3,2007,FL,PALM BEACH,32274.00
4,2007,FL,PALM BEACH,8068.50
...,...,...,...,...
15104978,2008,FL,HILLSBOROUGH,403425.00
15104979,2009,FL,HILLSBOROUGH,3027.00
15104980,2009,FL,HILLSBOROUGH,1513.50
15104981,2010,FL,HILLSBOROUGH,100856.25


In [11]:
final_shipments_FL[["Year"]] = final_shipments_FL["Year"].astype(int)

### CLEAN FLORIDA SHIPMENT DATA and CONCAT [unsure if we need to keep this section]

In [14]:
#Load in florida population DAta
fl_pop_pre_policy_03to09 = pd.read_csv("/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/fl_pre_policy_mortality.csv", index_col= False)
fl_pop_post_policy_10to15 = pd.read_csv("/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/fl_post_policy_mortality.csv", index_col= False)


fl_mortality = pd.concat([fl_pop_pre_policy_03to09,fl_pop_post_policy_10to15])



fl_mortality= fl_mortality.drop(['Unnamed: 0'], axis =1)

# make counties upper case for consistency
fl_mortality[['County']] = fl_mortality['County'].str.upper()
fl_mortality



,Year,State,County,Deaths,Population,Mortality Rate
0,2003,FL,ALACHUA COUNTY,11.0,227022.0,0.000048
1,2003,FL,BAY COUNTY,21.0,155044.0,0.000135
2,2003,FL,BREVARD COUNTY,97.0,504847.0,0.000192
3,2003,FL,BROWARD COUNTY,230.0,1707543.0,0.000135
4,2003,FL,CITRUS COUNTY,16.0,126215.0,0.000127
...,...,...,...,...,...,...
215,2015,FL,SEMINOLE COUNTY,63.0,448318.0,0.000250
216,2015,FL,ST. JOHNS COUNTY,17.0,226362.0,0.000222
217,2015,FL,ST. LUCIE COUNTY,51.0,297282.0,0.000226
218,2015,FL,SUMTER COUNTY,12.0,117076.0,0.000108


In [15]:


fl_pop = fl_mortality.drop(["State", "Deaths", "Mortality Rate"], axis=1)
fl_pop.head()

,Year,County,Population
0,2003,ALACHUA COUNTY,227022.0
1,2003,BAY COUNTY,155044.0
2,2003,BREVARD COUNTY,504847.0
3,2003,BROWARD COUNTY,1707543.0
4,2003,CITRUS COUNTY,126215.0


## MERGE FLORIDA SHIPMENT DATA WITH POPULATION DATA & FIPS CODE (MORTALITY DATA) 

### PIVOTED AWAY FROM USING FIPS CODE


In [27]:
fl_pre_pop = pd.read_csv("/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/fl_pre_population.csv", index_col= False)
fl_post_pop = pd.read_csv("/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/fl_post_population.csv", index_col= False)


fl_population = pd.concat([fl_pre_pop, fl_post_pop])





,Unnamed: 0,Year,County,Population
0,0,2003,Alachua County,227022.0
1,1,2003,Baker County,23555.0
2,2,2003,Bay County,155044.0
3,3,2003,Bradford County,27035.0
4,4,2003,Brevard County,504847.0
...,...,...,...,...
397,397,2015,Union County,15265.0
398,398,2015,Volusia County,516730.0
399,399,2015,Wakulla County,31523.0
400,400,2015,Walton County,63066.0


In [28]:
fl_population= fl_population.drop(['Unnamed: 0'], axis= 1) # drop unnamed column
fl_population[["County"]] = fl_population["County"].str.upper() #make entire column upper case 
fl_population['County'] = fl_population['County'].astype(str).str[:-7] # remove "county" from county name 
fl_population


,Year,County,Population
0,2003,ALACHUA,227022.0
1,2003,BAKER,23555.0
2,2003,BAY,155044.0
3,2003,BRADFORD,27035.0
4,2003,BREVARD,504847.0
...,...,...,...
397,2015,UNION,15265.0
398,2015,VOLUSIA,516730.0
399,2015,WAKULLA,31523.0
400,2015,WALTON,63066.0


In [38]:
fl_population

,Year,County,Population
0,2003,ALACHUA,227022.0
1,2003,BAKER,23555.0
2,2003,BAY,155044.0
3,2003,BRADFORD,27035.0
4,2003,BREVARD,504847.0
...,...,...,...
397,2015,UNION,15265.0
398,2015,VOLUSIA,516730.0
399,2015,WAKULLA,31523.0
400,2015,WALTON,63066.0


In [42]:
# # this merge isn't right but i kind of want to keep it 

# newdf = pd.merge(final_shipments_FL, fl_pop, on= ["County","Year"], how= 'outer', indicator= "True")
# newdf

In [51]:
newestdf = final_shipments_FL.merge(fl_population,  how='left')


In [52]:
newestdf

,Year,BUYER_STATE,County,MORPHINE_EQUIV_IN_MG,Population
0,2006,FL,PALM BEACH,5379.00,1284489.0
1,2006,FL,PALM BEACH,10758.00,1284489.0
2,2007,FL,PALM BEACH,32274.00,1286586.0
3,2007,FL,PALM BEACH,32274.00,1286586.0
4,2007,FL,PALM BEACH,8068.50,1286586.0
...,...,...,...,...,...
15104978,2008,FL,HILLSBOROUGH,403425.00,1196773.0
15104979,2009,FL,HILLSBOROUGH,3027.00,1214050.0
15104980,2009,FL,HILLSBOROUGH,1513.50,1214050.0
15104981,2010,FL,HILLSBOROUGH,100856.25,1233498.0


In [53]:
newestdf["MORPH_POP_RATIO"] = newestdf["MORPHINE_EQUIV_IN_MG"] / newestdf["Population"] 

In [54]:
newestdf


,Year,BUYER_STATE,County,MORPHINE_EQUIV_IN_MG,Population,MORPH_POP_RATIO
0,2006,FL,PALM BEACH,5379.00,1284489.0,0.004188
1,2006,FL,PALM BEACH,10758.00,1284489.0,0.008375
2,2007,FL,PALM BEACH,32274.00,1286586.0,0.025085
3,2007,FL,PALM BEACH,32274.00,1286586.0,0.025085
4,2007,FL,PALM BEACH,8068.50,1286586.0,0.006271
...,...,...,...,...,...,...
15104978,2008,FL,HILLSBOROUGH,403425.00,1196773.0,0.337094
15104979,2009,FL,HILLSBOROUGH,3027.00,1214050.0,0.002493
15104980,2009,FL,HILLSBOROUGH,1513.50,1214050.0,0.001247
15104981,2010,FL,HILLSBOROUGH,100856.25,1233498.0,0.081764
